In [1]:
import numpy as np
import pandas as pd
import datetime
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from datetime import timedelta

In [121]:
chrome_driver_path = '/Users/jackgoettle/Downloads/chromedriver'
browser = webdriver.Chrome(executable_path = chrome_driver_path)
base_url = 'https://www.fantasycruncher.com/contest-links/NBA/'

In [115]:
szn_start_2014 = datetime.datetime(2014, 10, 28)
szn_end_2014 = datetime.datetime(2015, 4, 15)
szn_start_2015 = datetime.datetime(2015, 10, 27)
szn_start_2016 = datetime.datetime(2016, 10, 25)
szn_end_2016 = datetime.datetime(2017, 4, 12)
szn_start_2017 = datetime.datetime(2017, 10, 17)
szn_start_2018 = datetime.datetime(2018, 10, 16)
szn_start_2019 = datetime.datetime(2019, 10, 22)
# convert attrbs to int
def to_int(num) :
    try :
        return int(num)
    except :
        return num

# convert attrbs to float
def to_float(num) :
    try :
        return float(num)
    except :
        return num

In [116]:
#load the page
def set_up(date, day) :
    url = base_url + date
    global browser
    browser.get(url)
    try : 
        browser.implicitly_wait(10)
         
        browser = fit_columns(browser, day)
        time.sleep(.25)
        browser.find_element_by_xpath('//*[@id="tournaments-list-grid"]/div/div[2]/div[1]/div[3]/div[2]/div/div/div[1]/div[12]').click()
    except :
        return None
    return browser

#resize the column and filter to just fanduel
def fit_columns(browser, day) :
    #resize
    time.sleep(.25)
    browser.find_element_by_class_name('grid-shrink-cols').click()
    time.sleep(.25)
    if day > datetime.datetime(2016, 11, 21) :
        #series of clicks to filter the data to just fanduel
        #browser.find_element_by_xpath('//*[@id="tournaments-list-grid"]/div/div[2]/div[1]/div[1]/div[2]/div/div/div[1]/div[3]/span').click()
        time.sleep(.25)
        #browser.find_element_by_xpath('//*[@id="tournaments-list-grid"]/div/div[6]/div/div/div[2]/div/div/div[1]/div[2]/div[1]/label').click()
        #time.sleep(.25)
        #browser.find_element_by_xpath('//*[@id="tournaments-list-grid"]/div/div[6]/div/div/div[2]/div/div/div[1]/div[2]/div[2]/div/div/div[1]').click()
        #time.sleep(.25)
    return browser

#load the table
def check_page(browser) : 
    table = browser.find_element_by_xpath('//*[@id="tournaments-list-grid"]/div/div[2]')
    if table == None :
        return False
    #print(table)
    return table



#scrape whatever is on the page
#added contests input so we could iteratively update contests while reloading the table
#ie pull contests out, click down, put contests back in, update table and append contests
def scrape1(browser, contests) :
    table = check_page(browser)
    values = table.text.split('\n')
    arr = [0] * 16
    print(values)
    print('------')
    count = 0
    offset = 0
    idx = 0
    loopbreak = 0
    while idx < len(values):
        if idx < 16 :
            idx += 1
            continue
        if values[idx] == 'Fanduel' and (idx + offset) % 16 != 0:
            if loopbreak > 100:
                print('loopbreak')
                break
            arr[12] = 'N/A'
            arr[13] = 'N/A'
            arr[14] = 'N/A'
            arr[15] = 'N/A'
            contest = { 'SITE':arr[0], 'STYLE':arr[1], 'LEAGUE':arr[2], 'PERIOD':arr[3], 'SLATE':arr[4],
                      'GAMES':arr[5],'CONTEST':arr[6], 'COST':arr[7], 'PRIZEPOOL':arr[8], 'LINEUPS':arr[9],
                        'MAX ENTRIES':arr[10], 'ACTUAL ENTRIES':arr[11], 'MIN CASH SCORE':arr[12],
                       'MIN CASH $':arr[13], '1ST PLACE SCORE':arr[14], '1ST PLACE $':arr[15] }
            #print(contest)
            count += 1
            #print(count)
            #print('OFFSET:' + str(offset))
            contests.append(contest)
            offset = offset + ((idx + offset) % 16)
            loopbreak += 2
            continue
        elif (idx + offset) % 16 == 0 :
            arr = [0] * 16
            arr[0] = values[idx]
        else :
            current_index = (idx + offset) % 16
            arr[current_index] = values[idx]
        if (idx + offset) % 16 == 15 and (idx + offset) != 15:
            contest = { 'SITE':arr[0], 'STYLE':arr[1], 'LEAGUE':arr[2], 'PERIOD':arr[3], 'SLATE':arr[4],
                      'GAMES':arr[5],'CONTEST':arr[6], 'COST':arr[7], 'PRIZEPOOL':arr[8], 'LINEUPS':arr[9],
                        'MAX ENTRIES':arr[10], 'ACTUAL ENTRIES':arr[11], 'MIN CASH SCORE':arr[12],
                       'MIN CASH $':arr[13], '1ST PLACE SCORE':arr[14], '1ST PLACE $':arr[15] }
            #print(contest)
            count += 1
            #print(count)
            contests.append(contest)
        idx += 1
    return contests


def scrape2(browser, contests) :
    table = check_page(browser)
    values = table.text.split('\n')
    count = 0
    idx = 0
    while idx < len(values):
        if idx <= 16 :
            idx += 1
            continue
        if values[idx] == 'No Rows To Show':
            break
        if values[idx] == 'Fanduel':
            try : 
                contest = { 'SITE':values[idx], 'STYLE':values[idx+1], 'LEAGUE':values[idx+2], 'PERIOD':values[idx+3], 
                           'SLATE':values[idx+5],'GAMES':values[idx+6],'CONTEST':values[idx+7], 'COST':values[idx+8], 
                           'PRIZEPOOL':values[idx+9], 'LINEUPS':values[idx+10],
                            'MAX ENTRIES':values[idx+11], 'ACTUAL ENTRIES':values[idx+12], 'MIN CASH SCORE':values[idx+13],
                           'MIN CASH $':values[idx+14], '1ST PLACE SCORE':values[idx+15], '1ST PLACE $':values[idx+16] }
                contests.append(contest)
            except : 
                break
            idx += 17
        else :
            idx += 1
    return contests


In [122]:
day =  datetime.datetime(2016,11,22)
end_just_fanduel = datetime.datetime(2016,11,21)
start_both = datetime.datetime(2016,11,22)
contests = []
count = 0
while day <= szn_end_2016 : 
    if day.date() in set(save_df['PERIOD']) :
        day = day + timedelta(days=1)
        continue
    print(day.date())
    break
    d = day.day
    date = str(day.year) + '-' + str(day.month) + '-' + str(day.day)
    set_up(date, day)
    if browser is None :
        day += timedelta(days=1)
        continue
    for idx in range(0, 8000, 500):  
        r = str(idx) + ', ' + str(idx + 500) + ')'
        browser.execute_script("document.querySelector('#tournaments-list-grid > div > div.ag-root-wrapper-body.ag-layout-normal > div.ag-root.ag-unselectable.ag-layout-normal > div.ag-body-viewport.ag-layout-normal.ag-row-no-animation').scrollTo(" + r)
        contests = scrape2(browser,contests)
    
    day = day + timedelta(days=1)
df = pd.DataFrame(contests)
df = df.drop_duplicates()

2016-11-22


In [137]:
d = datetime.datetime(2016,11,22)
d = pd.to_datetime(d)
print(d)
dates = save_df['PERIOD'] + ' '

2016-11-22 00:00:00


TypeError: unsupported operand type(s) for +: 'DatetimeIndex' and 'str'

In [131]:
set(save_df['PERIOD'])

{Timestamp('2016-11-22 00:00:00'),
 Timestamp('2016-11-23 00:00:00'),
 Timestamp('2016-11-25 00:00:00'),
 Timestamp('2016-11-26 00:00:00'),
 Timestamp('2016-11-27 00:00:00'),
 Timestamp('2016-11-28 00:00:00'),
 Timestamp('2016-11-29 00:00:00'),
 Timestamp('2016-11-30 00:00:00'),
 Timestamp('2016-12-01 00:00:00'),
 Timestamp('2016-12-02 00:00:00'),
 Timestamp('2016-12-03 00:00:00'),
 Timestamp('2016-12-04 00:00:00'),
 Timestamp('2016-12-05 00:00:00'),
 Timestamp('2016-12-06 00:00:00'),
 Timestamp('2016-12-07 00:00:00'),
 Timestamp('2016-12-08 00:00:00'),
 Timestamp('2016-12-09 00:00:00'),
 Timestamp('2016-12-10 00:00:00'),
 Timestamp('2016-12-11 00:00:00'),
 Timestamp('2016-12-12 00:00:00'),
 Timestamp('2016-12-13 00:00:00'),
 Timestamp('2016-12-14 00:00:00'),
 Timestamp('2016-12-15 00:00:00'),
 Timestamp('2016-12-16 00:00:00'),
 Timestamp('2016-12-17 00:00:00'),
 Timestamp('2016-12-18 00:00:00'),
 Timestamp('2016-12-19 00:00:00'),
 Timestamp('2016-12-20 00:00:00'),
 Timestamp('2016-12-

In [108]:
save_df = df.copy()

In [111]:
fanduel_all = fanduel_only.append(save_df)

In [2]:
df = None
with open('/Users/jackgoettle/DFS/dfs_payouts_2016.csv', 'r') as f :
    df = pd.read_csv(f)

In [113]:
fanduel_all.to_csv('fanduel-2016.csv', index = False)

In [133]:
fanduel_all

,SITE,STYLE,LEAGUE,PERIOD,SLATE,GAMES,CONTEST,COST,PRIZEPOOL,LINEUPS,MAX ENTRIES,ACTUAL ENTRIES,MIN CASH SCORE,MIN CASH $,1ST PLACE SCORE,1ST PLACE $
0,Fanduel,classic,NBA,2016-10-25,Main,3,$4K Tue NBA Block #4 (Single Entry),$2,"$4,000",1,2352,2352,303.2,$4,362.9,$400
1,Fanduel,classic,NBA,2016-10-25,Main,3,$4K Tue NBA Block #5 (Single Entry),$2,"$4,000",1,2352,2352,301.5,$4,354.4,$400
2,Fanduel,classic,NBA,2016-10-25,Main,3,$4K Tue NBA Block #6 (Single Entry),$2,"$4,000",1,2352,2352,301.6,$4,362.9,$400
3,Fanduel,classic,NBA,2016-10-25,Main,3,$4K Tue NBA BIG $5 Quintuple Up - 5X (Top 160 ...,$5,"$4,000",27,919,919,301.6,$25,351.9,$25
4,Fanduel,classic,NBA,2016-10-25,Main,3,$3300 Tue NBA BIG $11 Quintuple Up (Top 60 Win),$11,"$3,300",10,344,344,303.2,$55,346.9,$55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9132,Fanduel,classic,NBA,2017-03-29,Main,10,$2K NBA Mini Swat (Experienced Players Excluded),$1,"$2,000",25,2352,2352,313.6,$3,371.9,$200
9133,Fanduel,classic,NBA,2017-03-29,Express,3,$2K Wed NBA Block (Express),$2,"$2,000",1,1176,1176,284.1,$5,329.4,$200
9134,Fanduel,classic,NBA,2017-03-29,Express 2,2,$2K Wed NBA Block (Express 2),$2,"$2,000",1,1176,1176,310.1,$5,345.2,$200
9135,Fanduel,classic,NBA,2017-03-29,Late Night,3,$2K Wed NBA Block (Late Night),$2,"$2,000",1,1176,1176,284,$5,337.4,$200
